In [1]:
from adaptation import AdaptationHandler, create_adapted_module
from execution import execute_test
from stimulus_sheet_reader import get_stimulus_sheet
from solr_parser import parse_solr_response
from solr_query import translate_to_solr_query
from lql_parser import parse_interface_spec
import pysolr

In [2]:
lql_string = """
Calculator {
    Calculator(int)->None
    log(int, int)->float
    sqrd(int)->str
}
"""
interfaceSpecification = parse_interface_spec(lql_string)
print(interfaceSpecification)

InterfaceSpecification(className=Calculator, constructors=[MethodSignature(methodName=Calculator, returnType=None, parameterTypes=['int'])], methods=[MethodSignature(methodName=log, returnType=float, parameterTypes=['int', 'int']), MethodSignature(methodName=sqrd, returnType=str, parameterTypes=['int'])])


In [3]:
solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url)
solr_query = translate_to_solr_query(interfaceSpecification)
print("QUERY:", solr_query)
results = solr.search(solr_query)
print(f"Found {len(results)} results")

allModulesUnderTest = parse_solr_response(results)
moduleUnderTest = allModulesUnderTest[0]

QUERY: (name:log~0.1 AND (arguments.datatype:('int') AND arguments.datatype:('int'))) OR (name:log~0.1 AND count_positional_args:(2)) OR (name:sqrd~0.1 AND (arguments.datatype:('int'))) OR (name:sqrd~0.1 AND count_positional_args:(1)) OR (name:__init__ OR name:__new__ AND (arguments.datatype:('int'))) OR ((name:__init__ OR name:__new) AND count_positional_args:(1))&group=true&group.field=module
Found 10 results
numpy.lib.scimath
urllib3.connection
urllib3.response
numpy.distutils.ccompiler_opt
numpy.distutils.command.config
numpy.distutils.cpuinfo
Generated 6 ModuleUnderTest objects.


In [4]:
adaptationHandler = AdaptationHandler(interfaceSpecification, moduleUnderTest, excludeClasses=False, useFunctionDefaultValues=False)
adaptationHandler.identifyAdaptations(maxParamPermutationTries=2)
adaptationHandler.visualizeAdaptations()


-----------------
IDENTIFY MAPPINGS
-----------------

                  sqrt                            logn
log              None  [NameRtrnConvr, NameRtrnPerm*]
sqrd  [NameRtrnConvr]                            None 



In [5]:
adaptationHandler.generateMappings(onlyKeepTopN=10)

Generated 2 potential mappings.
[log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | score=16
[log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | score=9


In [6]:
(adapted_module, successful_mappings)  = create_adapted_module(adaptationHandler, moduleUnderTest.moduleName, use_constructor_default_values=True)


-----------------------------
TRYING ADAPTATION FOR MAPPING [log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | score=16.
-----------------------------
Trying to adapt return type of <function logn at 0x7fe246d765e0> to float.
Trying to blindly adapt parameter order of <function logn at 0x7fe246d765e0>.
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fe24868ad30> to log.
Trying to adapt return type of <function sqrt at 0x7fe246d763a0> to str.
Trying to adapt parameter types of <function sqrt at 0x7fe246d763a0> to ['Any'].
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fe24868aee0> to sqrd.
Successful creation of submodule 0 for this mapping.

-----------------------------
TRYING ADAPTATION FOR MAPPING [log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | score=9.
-----------------------------
Trying to adapt retu

In [7]:
stimulus_sheet = get_stimulus_sheet("calc4.csv")
execute_test(stimulus_sheet, adapted_module, successful_mappings)


----------------------
EXECUTE STIMULUS SHEET
----------------------
Module: numpy.lib.scimath

    output_param method_name  instance_param input_params
1         343.0        sqrd             NaN        [7.0]
2           1.0         log             NaN   [3.0, 2.0]

0 [log->logn via NameRtrnPerm*][sqrd->sqrt via NameRtrnConvr] | score=16	
	sqrd(7.0): 2.6457513110645907, Lines: 6 total, 2/3 in function. Arcs: 9 total, 3 in function.
	log(3.0, 2.0): 1.5849625007211563, Lines: 7 total, 3/4 in function. Arcs: 10 total, 4 in function.
1 [log->logn via NameRtrnConvr][sqrd->sqrt via NameRtrnConvr] | score=9	
	sqrd(7.0): 2.6457513110645907, Lines: 6 total, 2/3 in function. Arcs: 9 total, 3 in function.
	log(3.0, 2.0): 0.6309297535714574, Lines: 7 total, 3/4 in function. Arcs: 10 total, 4 in function.
